In [10]:
# ==== 0. INSTALL LIBRARY ====
!pip install tensorflow==2.19.1 mediapipe==0.10.7 protobuf==3.20.3 opencv-python scikit-learn matplotlib seaborn tqdm --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.19.1 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.19.1 which is incompatible.


In [ ]:
# ==== 1. IMPORTS & SETUP ====
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import joblib

mp_hands = mp.solutions.hands

In [ ]:
# ==== 2. EKSTRAKSI FITUR ====
def extract_hand_vector_advanced(landmarks):
    """
    Ekstraksi fitur jarak & sudut antar jari.
    Output: 19 dimensi vektor.
    """
    base = landmarks[0]
    rel = landmarks - base
    palm_size = np.linalg.norm(rel[0] - rel[9]) + 1e-6

    #  Jarak ibu jari ke jari lain
    d_thumb = [np.linalg.norm(rel[4] - rel[i]) / palm_size for i in [8, 12, 16, 20]]

    #  Panjang tiap jari
    finger_len = [np.linalg.norm(rel[0] - rel[i]) / palm_size for i in [4, 8, 12, 16, 20]]

    # Sudut antar ruas (cosine angle)
    def angle(a, b, c):
        ba, bc = a - b, c - b
        cosang = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc) + 1e-6)
        return np.arccos(np.clip(cosang, -1, 1))

    angs = [
        angle(rel[2], rel[3], rel[4]),
        angle(rel[5], rel[6], rel[8]),
        angle(rel[9], rel[10], rel[12]),
        angle(rel[13], rel[14], rel[16]),
        angle(rel[17], rel[18], rel[20])
    ]

    return np.array(d_thumb + finger_len + angs)

In [3]:
# ==== 3. AUGMENTASI FITUR ====
def augment_vector(vec, n_aug=3, noise=0.01):
    aug = []
    for _ in range(n_aug):
        noise_vec = vec + np.random.normal(0, noise, size=vec.shape)
        aug.append(noise_vec)
    return aug

In [ ]:
# ==== 4. EKSTRAKSI DATASET ====
dataset_dir = "./SIBI/alphabet"   # ubah sesuai folder dataset kamu
output_dir = "./SIBI/alphabet_vector"
os.makedirs(output_dir, exist_ok=True)

labels_list = sorted(os.listdir(dataset_dir))
label_map = {label: i for i, label in enumerate(labels_list)}

X, y = [], []

with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.6) as hands:
    for label in tqdm(labels_list, desc="Ekstraksi dataset"):
        folder = os.path.join(dataset_dir, label)
        for file in os.listdir(folder):
            img_path = os.path.join(folder, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            result = hands.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
            if not result.multi_hand_landmarks:
                continue

            hand = result.multi_hand_landmarks[0]
            pts = np.array([[lm.x, lm.y, lm.z] for lm in hand.landmark])
            vec = extract_hand_vector_advanced(pts)

            X.append(vec)
            y.append(label_map[label])

            # augmentasi ringan
            for aug_vec in augment_vector(vec):
                X.append(aug_vec)
                y.append(label_map[label])

X = np.array(X)
y = np.array(y)


Ekstraksi dataset:   0%|          | 0/24 [00:00<?, ?it/s]

Ekstraksi dataset: 100%|██████████| 24/24 [04:06<00:00, 10.28s/it]


In [ ]:
# ==== 4. SIMPAN DATASET VECTOR ====
np.save(os.path.join(output_dir, "X_vectors.npy"), X)
np.save(os.path.join(output_dir, "y_labels.npy"), y)
np.save(os.path.join(output_dir, "label_map.npy"), label_map)

print(f"   Dataset vektor tersimpan di: {output_dir}")
print(f"   Jumlah sampel: {len(X)} | Fitur: {X.shape[1]} | Kelas: {len(labels_list)}")

✅ Dataset vektor tersimpan di: ./SIBI/vector_alphabet
   Jumlah sampel: 9332 | Fitur: 14 | Kelas: 24


In [ ]:
# ==== 5. LOAD DATASET ====
X = np.load("./SIBI/alphabet_vector/X_vectors.npy")
y = np.load("./SIBI/alphabet_vector/y_labels.npy")
label_map = np.load("./SIBI/alphabet_vector/label_map.npy", allow_pickle=True).item()
labels_list = list(label_map.keys())

print(" Data loaded:", X.shape, " | Classes:", labels_list)

✅ Data loaded: (9332, 14)  | Classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']


In [9]:
# ==== 6. SPLIT TRAIN-TEST ====
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


In [ ]:
# ==== 7. TRAIN MODEL ====
model = MLPClassifier(hidden_layer_sizes=(128, 64, 32),
                      activation='relu',
                      solver='adam',
                      max_iter=1000,
                      random_state=42)
model.fit(X_train, y_train)


MLPClassifier(hidden_layer_sizes=(128, 64, 32), max_iter=1000, random_state=42)

In [ ]:
# ==== 8. EVALUASI ====
y_pred = model.predict(X_test)
print("\n Akurasi:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=labels_list))



🎯 Akurasi: 0.9448312801285484
              precision    recall  f1-score   support

           A       1.00      1.00      1.00       107
           B       1.00      1.00      1.00        95
           C       0.98      0.95      0.97        63
           D       0.95      1.00      0.97        77
           E       0.95      0.98      0.97        99
           F       0.99      0.99      0.99        87
           G       0.99      0.96      0.97        81
           H       0.74      0.77      0.75        83
           I       1.00      1.00      1.00        87
           K       0.89      0.97      0.93        66
           L       1.00      1.00      1.00        75
           M       1.00      0.93      0.96        87
           N       0.95      1.00      0.98        84
           O       1.00      0.97      0.98        60
           P       0.89      0.79      0.84        42
           Q       0.90      0.95      0.92        38
           R       0.95      0.75      0.84       

In [ ]:
# ==== 9. SIMPAN MODEL ====
os.makedirs("model_vector", exist_ok=True)
joblib.dump(model, "model_vector/mauna_alphabet_model.pkl")
np.save("model_vector/mauna_alphabet_label_map.npy", label_map)
print("Model disimpan di model_vector/")

💾 Model disimpan di model_vector/


In [1]:
import cv2, mediapipe as mp, time, numpy as np, joblib
from collections import deque

mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

model = joblib.load("model_vector/mauna_alphabet_model.pkl")
label_map = np.load("model_vector/mauna_alphabet_label_map.npy", allow_pickle=True).item()
inv_label = {v:k for k,v in label_map.items()}

def extract_hand_vector_advanced(landmarks):
    base = landmarks[0]
    rel = landmarks - base
    palm_size = np.linalg.norm(rel[0] - rel[9]) + 1e-6
    d_thumb = [np.linalg.norm(rel[4] - rel[i]) / palm_size for i in [8, 12, 16, 20]]
    finger_len = [np.linalg.norm(rel[0] - rel[i]) / palm_size for i in [4, 8, 12, 16, 20]]
    def angle(a,b,c):
        ba, bc = a-b, c-b
        cosang = np.dot(ba, bc) / (np.linalg.norm(ba)*np.linalg.norm(bc)+1e-6)
        return np.arccos(np.clip(cosang, -1, 1))
    angs = [angle(rel[2], rel[3], rel[4]), angle(rel[5], rel[6], rel[8]), angle(rel[9], rel[10], rel[12]), angle(rel[13], rel[14], rel[16]), angle(rel[17], rel[18], rel[20])]
    return np.array(d_thumb + finger_len + angs)

cap = cv2.VideoCapture(0)
smooth_labels = deque(maxlen=10)
pred_label, pred_conf = "...", 0.0
last_time = 0

with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7) as hands:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        now = time.time()

        if now - last_time > 0.5:
            last_time = now
            result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if result.multi_hand_landmarks:
                pts = np.array([[lm.x, lm.y, lm.z] for lm in result.multi_hand_landmarks[0].landmark])
                vec = extract_hand_vector_advanced(pts)
                probs = model.predict_proba([vec])[0]
                idx = np.argmax(probs)
                label = inv_label[idx]
                conf = probs[idx]

                smooth_labels.append(label)
                pred_label = max(set(smooth_labels), key=smooth_labels.count)
                pred_conf = conf

        # draw hand
        result = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if result.multi_hand_landmarks:
            for hand in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

        cv2.putText(frame, f"{pred_label} ({pred_conf*100:.1f}%)", (30,80),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255,0), 3)
        cv2.imshow("SIBI Alphabet - Realtime Vector", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()
